<a href="https://colab.research.google.com/github/fjadidi2001/Artificial_Intelligence_Learning/blob/master/Alzheimer's_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Explanation:
<br>

## architecture:

> This project presents a CNN architecture that uses MRI data from the ADNI dataset to classify Alzheimer’s disease. The network leverages different CNN models, filter sizes, and pooling layers to extract features from MRI images, enabling precise classification of AD subtypes and stages. Accurate classification can aid in early detection, personalized treatment planning, disease monitoring, and prognostic assessment, showcasing the potential of this CNN architecture to assist medical professionals and researchers in making informed decisions about AD patients.



## what is Alzheimer's disease (AD)?

> Alzheimer's disease (AD) is a common brain ailment leading to memory loss and cognitive decline, ranking as a significant cause of mortality.

## what is MCI?

> Mild cognitive impairment (MCI) lies between normal brain function and AD, with AD progressing from the prodromal stage of MCI to dementia. Identifying MCI early can help delay or prevent the progression to AD, with studies showing a 10-15% annual development rate from MCI to AD. Intermediate MCI stages show subtle brain lesion variations.



> Early mild cognitive impairment (EMCI) emerges in the initial phase of MCI, while late mild cognitive impairment (LMCI) or progressive MCI (PMCI) indicates worsening symptoms over time. As symptoms evolve, transitioning between stages requires careful monitoring by medical professionals. Researchers face challenges in identifying distinct symptoms across different sets. Medical imaging techniques like PET, MRI, and CT provide crucial standardized testing and images for experimental processes in this field.



## Features extracted from MRI

> MRI is a widely recognized and versatile tool for diagnosing various diseases such as brain tumors, neurological disorders, spinal cord injuries, and liver diseases. Its high sensitivity allows for early disease detection. Different MRI sequences are suited for different disorders, and MRI images are frequently used for Alzheimer's disease classification. Features extracted from MRI images, **such as grey and white matter volumes, cortical thickness, and CSF levels, aid in categorizing and diagnosing mild cognitive impairment or Alzheimer's disease**. Pre-trained CNNs show promise in automatically diagnosing cognitive illnesses from brain MRI images.

# connect to drive

In [13]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [14]:
# Path to your dataset
dataset_path = '/content/drive/My Drive/Deep learning/Alzheimer_s_Disease_Neuroimaging_ADNI_Dataset'

# import libraries

In [21]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
import cv2
import random
from sklearn.preprocessing import StandardScaler
import numpy as np
import pandas as pd


# split dataset and normalize images

In [23]:
# Path to the directories containing the AD and MCI images
ad_path = '/content/drive/My Drive/Deep learning/Alzheimer_s_Disease_Neuroimaging_ADNI_Dataset/AD'
mci_path = '/content/drive/My Drive/Deep learning/Alzheimer_s_Disease_Neuroimaging_ADNI_Dataset/MCI'

# Function to get the list of image file paths in a directory
def get_image_paths(directory):
    return [os.path.join(directory, image) for image in os.listdir(directory)]

# Get the list of image file paths for AD and MCI images
ad_images = get_image_paths(ad_path)
mci_images = get_image_paths(mci_path)

# Combine image paths with labels
ad_labels = np.zeros(len(ad_images))
mci_labels = np.ones(len(mci_images))

# Combine all images and labels
all_images = ad_images + mci_images
all_labels = np.concatenate([ad_labels, mci_labels])

# Split the dataset into training, testing, and validation sets
X_train, X_test, y_train, y_test = train_test_split(all_images, all_labels, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# Normalize the images (You should replace this with your actual normalization code)
def normalize_image(image_path):
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert image to RGB
    image = image / 255.0  # Normalize pixel values between 0 and 1
    return image_path



X_train = [normalize_image(image) for image in X_train]
X_test = [normalize_image(image) for image in X_test]
X_val = [normalize_image(image) for image in X_val]

# Check the distribution of classes in each split
print("Training set - AD: {}, MCI: {}".format(np.sum(y_train == 0), np.sum(y_train == 1)))
print("Testing set - AD: {}, MCI: {}".format(np.sum(y_test == 0), np.sum(y_test == 1)))
print("Validation set - AD: {}, MCI: {}".format(np.sum(y_val == 0), np.sum(y_val == 1)))

Training set - AD: 609, MCI: 455
Testing set - AD: 204, MCI: 129
Validation set - AD: 162, MCI: 105


In [24]:


# Function for resizing and augmenting images
def preprocess_image(image_path, target_size=(224, 224)):
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert image to RGB
    image = cv2.resize(image, target_size)  # Resize the image to the target size

    # Data augmentation (you can add more augmentation techniques)
    if random.random() < 0.5:
        image = cv2.flip(image, 1)  # Horizontal flip with 50% probability
    # Add more augmentation techniques as needed

    return image

# Preprocess all images in the datasets
X_train = [preprocess_image(image) for image in X_train]
X_test = [preprocess_image(image) for image in X_test]
X_val = [preprocess_image(image) for image in X_val]